In [102]:
import joblib
import pandas as pd
import numpy as np
import re

from pythainlp import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.metrics import classification_report, confusion_matrix


In [103]:

# Function to detect language and apply correct tokenization
def custom_tokenizer(text):
    # Remove special characters & normalize spaces
    text = re.sub(r'[^\w\s]', '', text).strip().lower()
    
    # If Thai characters are present, use Thai tokenizer
    if any("\u0E00" <= char <= "\u0E7F" for char in text):
        return word_tokenize(text)  # Thai word segmentation
    else:
        return text.split()  # Simple English tokenization


In [104]:

augmented_data = {
    "text": [
        # Factory Automation (40)
        "Robotic welding arms increase automotive assembly speed",
        "PLC ladder logic programming for conveyor synchronization",
        "Automated packaging systems using delta robots",
        "HMI design principles for production line monitoring",
        "Industrial robot calibration using laser alignment",
        "Machine tending automation with collaborative robots",
        "Automated optical inspection (AOI) for PCB assembly",
        "Palletizing robot programming with ROS framework",
        "Industrial automation using OPC UA communication",
        "CNC machine integration with MES systems",
        "Automated guided vehicle navigation using LiDAR",
        "Pick-and-place robots in electronics manufacturing",
        "Automated storage and retrieval systems (AS/RS)",
        "Industrial robot end-effector tooling design",
        "Production line balancing using automation software",
        "Automated lubrication systems for machinery",
        "Machine vision guided robotic assembly",
        "Industrial automation network segmentation",
        "Automated quality assurance through force sensing",
        "Robotic deburring applications in machining",
        "Automated screwdriving systems for appliance assembly",
        "Vision-guided robotic bin picking solutions",
        "Automated leak testing systems for quality control",
        "Industrial robot offline programming techniques",
        "Automated conveyor belt alignment procedures",
        "Robotic painting systems with adaptive path planning",
        "Automated fastener verification using torque sensors",
        "Industrial cobot payload capacity optimization",
        "Automated dimensional inspection with CMMs",
        "Robotic waterjet cutting cell integration",
        "Automated tool changing systems for CNC",
        "Production line OEE monitoring systems",
        "Automated soldering systems for PCB assembly",
        "Robotic adhesive dispensing cell calibration",
        "Automated stamping press feeding mechanisms",
        "Industrial robot collision detection systems",
        "Automated vision-based sorting conveyors",
        "Robotic arc welding seam tracking systems",
        "Automated press-fit assembly validation",
        "Industrial automation cabinet cooling solutions",

        # IoT (40)
        "Wireless vibration sensors for motor health monitoring",
        "IoT-enabled smart valves in water treatment plants",
        "BLE mesh networks for factory equipment tracking",
        "IoT data ingestion pipelines using Apache Kafka",
        "Predictive maintenance through thermal imaging sensors",
        "LoRaWAN gateways for long-range sensor networks",
        "Digital twin implementation for HVAC systems",
        "IoT device management using Azure Sphere",
        "Wireless pressure monitoring in hydraulic systems",
        "Edge AI processing for vibration analytics",
        "MQTT brokers in industrial IoT architectures",
        "Energy monitoring using smart IoT meters",
        "Vibration spectrum analysis for bearing health",
        "IoT security protocols for device authentication",
        "RFID-based inventory tracking in warehouses",
        "Wireless tank level sensors with NB-IoT",
        "Predictive quality analytics using sensor fusion",
        "IoT-enabled smart gloves for worker safety",
        "5G private networks for factory IoT devices",
        "Condition monitoring using ultrasonic sensors",
        "IoT-enabled predictive corrosion monitoring",
        "Wireless gas detection sensors with LTE-M",
        "Digital shadow implementation for pumps",
        "IoT-based energy harvesting sensor nodes",
        "Vibration-based gearbox health monitoring",
        "IoT-enabled smart bearing wear detection",
        "Wireless acoustic emission monitoring",
        "IoT-driven compressed air leak detection",
        "Smart lighting control using occupancy sensors",
        "IoT-enabled steam trap monitoring systems",
        "Wireless flowmeter integration with SCADA",
        "IoT-based lubrication level monitoring",
        "Smart electrical panel thermal monitoring",
        "IoT-enabled valve position feedback systems",
        "Wireless strain gauge data acquisition",
        "IoT-based hydraulic fluid quality sensing",
        "Smart motor current signature analysis",
        "IoT-enabled conveyor belt wear monitoring",
        "Wireless proximity sensor network design",

        # Process Control (40)
        "PID loop tuning methods for heat exchangers",
        "Distributed control system (DCS) architecture",
        "Advanced process control for distillation columns",
        "Mass flow controller calibration techniques",
        "Model predictive control in chemical reactors",
        "Pressure control loops in steam systems",
        "Cascade control configuration for reactors",
        "Statistical process control (SPC) chart analysis",
        "Flow transmitter installation best practices",
        "Control valve positioner configuration",
        "Batch process automation in pharma industry",
        "Multi-variable control for furnace operation",
        "Feedforward control for disturbance rejection",
        "Level control strategies for separator vessels",
        "Process historian data analysis trends",
        "Control loop performance monitoring tools",
        "Safety instrumented systems in process plants",
        "PID anti-windup techniques implementation",
        "Process variable filtering and signal damping",
        "Ratio control systems for mixing processes",
        "Advanced regulatory control configurations",
        "Split-range control valve methodologies",
        "Process gain calculation for controller tuning",
        "Override control strategies for safety",
        "Adaptive control for nonlinear processes",
        "Dead-time compensation using Smith predictors",
        "Process reaction curve analysis methods",
        "Valve stiction detection and compensation",
        "Relative gain analysis for loop pairing",
        "Inferential control using soft sensors",
        "Process decoupling control techniques",
        "Real-time optimization (RTO) integration",
        "Surge control for centrifugal compressors",
        "Boiler drum level control strategies",
        "pH neutralization process control methods",
        "Distributed temperature control systems",
        "Exothermic reactor temperature control",
        "Flooded condenser control in distillation",
        "Reboiler steam flow control optimization",
        "Pump minimum flow protection controls",

        # Safety and Standards (40)
        "ISO 13849 PL ratings for machine safety",
        "Safety door interlock circuit design",
        "IEC 62061 functional safety compliance",
        "ATEX zone classification for hazardous areas",
        "Machine safety risk assessment methodology",
        "Safety relay configuration for E-stop circuits",
        "ANSI B11 standards for press brake safety",
        "SIL verification for safety instrumented systems",
        "Lockout-tagout (LOTO) procedure compliance",
        "Emergency stop circuit design principles",
        "Safety light curtain integration guidelines",
        "NFPA 79 electrical standards for machinery",
        "Functional safety certification process",
        "Safety PLC programming standards (IEC 61131)",
        "Risk reduction factors in machine guarding",
        "CE marking requirements for industrial equipment",
        "Safety network protocols (CIP Safety, PROFIsafe)",
        "Hazardous area classification per NEC 500",
        "Safety validation documentation requirements",
        "Human-machine interface safety standards",
        "Safety mat installation and configuration",
        "Two-hand control system implementation",
        "Emergency stop button ergonomic placement",
        "Safety-rated monitored stop systems",
        "ISO 10218 robot safety compliance",
        "Safety controller response time analysis",
        "Machine safety distance calculations",
        "Safety encoder integration for position monitoring",
        "Safety over Ethernet protocols implementation",
        "Pilz safety relay configuration guidelines",
        "Safety laser scanner zone configuration",
        "ISO 12100 risk assessment procedures",
        "Safety contactor selection criteria",
        "Emergency stop circuit redundancy design",
        "Safety interlock switch mounting standards",
        "Safety control cabinet layout requirements",
        "ISO 13857 reach distance regulations",
        "Safety function performance level verification",
        "Safe torque off (STO) circuit design",
        "Safety limit switch installation standards",

        # General (40)
        "How to update my account password?",
        "Where can I find my purchase history?",
        "The website is not loading images properly",
        "How to contact customer support team?",
        "Need help recovering my account email",
        "Why am I getting payment gateway errors?",
        "How to cancel my subscription plan?",
        "Download link not working for manuals",
        "Two-factor authentication setup guide",
        "Invoice format customization options",
        "System requirements for mobile app",
        "Request for VAT exemption documents",
        "Trouble accessing online training portal",
        "How to change account permissions?",
        "Software license transfer procedure",
        "Report inappropriate forum content",
        "Data export format options available",
        "Request extension for service trial",
        "Update billing address information",
        "Troubleshoot notification delays",
        "How to merge duplicate user accounts?",
        "Where are my saved configuration files?",
        "Error when uploading profile picture",
        "How to request API documentation?",
        "Trouble printing from web interface",
        "How to reset dashboard preferences?",
        "Mobile app crashing on startup",
        "Request product compatibility matrix",
        "How to share project access?",
        "Trouble redeeming license key",
        "Where to find software checksums?",
        "How to report a translation error?",
        "Request archived report generation",
        "Trouble with search functionality",
        "How to enable dark mode theme?",
        "Missing toolbar icons in interface",
        "How to request training materials?",
        "Trouble with auto-save feature",
        "How to check service uptime?",
        "Request custom report templates",
        "Request custom report templates"
    ],
    
    "category": (
        *["Factory Automation"] * 40 ,
        *["IoT"] * 40 ,
        *["Process Control"] * 40 ,
        *["Safety and Standards"] * 40 ,
        *["General"] * 40
    )
}
first_data = pd.DataFrame(augmented_data)

In [105]:

additional_data = {
    "text": [
        # Factory Automation (+20)
        "Automated CNC tool changers for machining centers",
        "Vision-guided robotic inspection of weld seams",
        "Automated labeling systems with print-and-apply technology",
        "Industrial robot singularity avoidance algorithms",
        "Automated leak testing using pressure decay methods",
        "Robotic arc welding parameter optimization techniques",
        "Automated fixture calibration for assembly stations",
        "Conveyor belt tracking adjustment using laser alignment",
        "Automated spindel runout compensation systems",
        "Robotic ultrasonic cleaning cell integration",
        "Automated coordinate system calibration for robots",
        "Industrial robot payload dynamic compensation",
        "Automated glue dispensing system nozzle maintenance",
        "Machine vision-based robot workcell monitoring",
        "Automated press brake tooling configuration",
        "Robotic sanding path optimization algorithms",
        "Automated coolant concentration monitoring systems",
        "Industrial robot collision avoidance zone setup",
        "Automated vision-based part orientation detection",
        "Robotic gripper force feedback calibration",

        # IoT (+20)
        "Wireless corrosion monitoring using IoT sensors",
        "IoT-enabled predictive bearing failure detection",
        "BLE-based personnel tracking in restricted areas",
        "IoT-driven pump cavitation detection systems",
        "Wireless torque sensor networks for rotating equipment",
        "IoT-based hydraulic system contamination monitoring",
        "LoRa-enabled outdoor tank level monitoring",
        "Edge computing for vibration frequency analysis",
        "IoT-enabled motor current signature analysis",
        "Wireless infrared thermography for electrical panels",
        "MQTT-SN protocol for constrained IoT devices",
        "IoT-based steam trap efficiency monitoring",
        "Wireless strain gauge networks on structural beams",
        "Digital shadow implementation for compressor health",
        "IoT-enabled lubricant viscosity monitoring",
        "Wireless partial discharge detection in switchgear",
        "NB-IoT based environmental monitoring in plants",
        "Edge AI for abnormal sound pattern detection",
        "IoT-driven predictive valve packing maintenance",
        "Wireless encoder feedback monitoring systems",

        # Process Control (+20)
        "Split-range control for heating/cooling systems",
        "Adaptive PID tuning using process gain scheduling",
        "Cascade control for reactor temperature management",
        "Dead-band adjustment in level control systems",
        "Ratio control for chemical dosing applications",
        "Override control for pump protection systems",
        "Feedforward control for feed temperature variations",
        "Nonlinear control strategies for pH regulation",
        "Model-based predictive control for distillation",
        "Anti-surge control for centrifugal compressors",
        "Constraint handling in advanced process control",
        "Valve position tracking for stiction detection",
        "Multivariable control for furnace pressure/temp",
        "Self-tuning regulators for process variability",
        "Gain margin analysis in control loop design",
        "Real-time optimization of cracking furnaces",
        "Inferential control using Raman spectroscopy",
        "Adaptive filtering for noisy process variables",
        "Decoupling control for interacting processes",
        "Bumpless transfer between manual/auto modes",

        # Safety and Standards (+20)
        "ISO 14119 interlock device safety requirements",
        "Safety mat response time validation procedures",
        "IEC 61496 safety light curtain classifications",
        "ATEX Category 2 equipment installation guidelines",
        "Safety controller watchdog timer configurations",
        "ISO 13855 safety distance calculation formulas",
        "Emergency stop circuit redundancy verification",
        "Safety PLC code compliance auditing procedures",
        "EN 62061 subsystem architecture requirements",
        "Lockout tagout procedure documentation standards",
        "Safety relay force-guided contact verification",
        "ISO 13857 machine reach distance regulations",
        "Safety function diagnostic coverage analysis",
        "EN 1037 machinery unexpected start prevention",
        "Safety network heartbeat monitoring protocols",
        "IEC 60204 electrical equipment safety checks",
        "Safety-rated encoder signal validation",
        "PLr (Performance Level required) determination",
        "Safety controller validation test procedures",
        "Emergency stop circuit voltage drop analysis",

        # General (+20)
        "How to export my transaction history to CSV?",
        "Why are my notification settings resetting?",
        "Trouble accessing archived project files",
        "How to request a certificate of compliance?",
        "Mobile app compatibility with Android 14",
        "Procedure to merge duplicate contacts",
        "How to reset two-factor authentication?",
        "Download speed issues with large files",
        "Request custom API rate limit increase",
        "Trouble applying discount codes at checkout",
        "How to enable keyboard shortcuts?",
        "Missing email attachments in web client",
        "Request historical audit trail reports",
        "How to change account timezone settings?",
        "Trouble with auto-renewal cancellation",
        "Procedure to escalate support tickets",
        "How to access previous software versions?",
        "Calendar sync issues with mobile app",
        "Request custom data retention policies",
        "How to report broken documentation links?"
    ],
    
    "category": [
        *["Factory Automation"]*20,
        *["IoT"]*20,
        *["Process Control"]*20,
        *["Safety and Standards"]*20,
        *["General"]*20
    ]
}
second_data = pd.DataFrame(additional_data)

In [106]:
additional_data = {
    "text": [
        # Factory Automation (+20)
        "Robotic polishing cells with adaptive force control",
        "Automated coordinate measuring machine (CMM) integration",
        "Vision-guided robotic deburring of cast parts",
        "Automated tool life monitoring using acoustic emissions",
        "Robotic adhesive application path optimization",
        "Automated guided vehicle (AGV) traffic management systems",
        "Machine vision-based robotic sorting systems",
        "Automated spindle temperature compensation algorithms",
        "Robotic welding seam tracking with laser sensors",
        "Automated fixtureless clamping systems",
        "Industrial robot singularity avoidance algorithms",
        "Automated coolant filtration system monitoring",
        "Robotic laser cutting head focal distance adjustment",
        "Automated press tonnage monitoring systems",
        "Robot-mounted 3D scanning for inline inspection",
        "Automated vacuum gripper pressure monitoring",
        "Robotic assembly error proofing techniques",
        "Automated conveyor belt tension regulation",
        "Machine tending robots with dual gripper systems",
        "Automated vision-based fastener verification",

        # IoT (+20)
        "Wireless motor current signature analysis",
        "IoT-enabled bearing temperature trend monitoring",
        "LoRa-based pipeline cathodic protection monitoring",
        "Edge computing for pump performance degradation alerts",
        "Wireless ultrasonic thickness measurement systems",
        "IoT-driven predictive refractory wear monitoring",
        "BLE mesh networks for factory environmental sensing",
        "Vibration-based gearbox fault prediction models",
        "IoT-enabled steam system trap failure detection",
        "Wireless power quality monitoring in distribution panels",
        "NB-IoT enabled fire detection in remote facilities",
        "Edge AI for predictive conveyor belt wear",
        "IoT-based compressed air moisture content monitoring",
        "Wireless thermocouple networks for furnace profiling",
        "Digital twin integration for pump system optimization",
        "IoT-enabled predictive grease lubrication scheduling",
        "Wireless dissolved oxygen monitoring in process water",
        "LPWAN-based tank farm overfill protection",
        "Edge computing for real-time bearing FFT analysis",
        "IoT-driven predictive heat exchanger fouling",

        # Process Control (+20)
        "Model predictive control for batch reactors",
        "Adaptive PID for non-linear heat transfer processes",
        "Cascade control for distillation column pressure",
        "Ratio control optimization in polymerization",
        "Override control for reactor pressure relief",
        "Feedforward compensation for feed composition changes",
        "Dead-time dominant process Smith predictor design",
        "Multivariable control for extractive distillation",
        "Robust control for varying raw material properties",
        "Inferential control using near-infrared spectroscopy",
        "Constraint handling in ethylene cracking furnaces",
        "Valve positioner tuning for fast response loops",
        "Nonlinear model predictive control for pH swings",
        "Adaptive filtering for noisy level measurements",
        "Decoupling control for interactive temperature zones",
        "Bumpless transfer in split-range control systems",
        "Real-time optimization of steam methane reformers",
        "Fault-tolerant control system architecture design",
        "Gain scheduling for seasonal cooling variations",
        "Anti-windup strategies for integral saturation",

        # Safety and Standards (+20)
        "IEC 62061 safety function response time analysis",
        "ISO 13849-1 architecture category validation",
        "Safety laser scanner configuration for AGV paths",
        "ATEX Zone 0 equipment certification requirements",
        "Emergency stop circuit impedance testing",
        "Safety controller cross-checking methodologies",
        "ISO 14123 machinery hazardous substance reduction",
        "Safety mat system fault injection testing",
        "EN ISO 10218-2 robot safety standard compliance",
        "Lockout tagout device durability testing",
        "Safety PLC program sequence verification",
        "ISO 13856 pressure-sensitive mat specifications",
        "Safety function performance level (PL) verification",
        "Emergency stop button ergonomic placement guidelines",
        "Safety relay contact welding detection methods",
        "IEC 61496 electro-sensitive protective equipment",
        "Machine safety risk graph methodology application",
        "Safety network communication integrity checks",
        "PL (Performance Level) calculation worksheets",
        "Emergency stop circuit voltage sag immunity tests",

        # General (+20)
        "How to recover deleted user permissions?",
        "Troubleshooting API authentication failures",
        "Mobile app notification delay issues",
        "Requesting historical access logs",
        "How to transfer project ownership?",
        "Resolving certificate expiration warnings",
        "Two-step verification backup code recovery",
        "Exporting system configuration backups",
        "Trouble accessing legacy API versions",
        "How to request custom report templates?",
        "Password complexity rule configuration",
        "Troubleshooting dashboard data latency",
        "Requesting SCORM package exports",
        "How to enable dark mode in web interface?",
        "Resolving time zone synchronization issues",
        "Procedure to archive inactive projects",
        "How to generate usage trend reports?",
        "Troubleshooting CSV import formatting",
        "Requesting increased file storage quota",
        "How to report translation errors in UI?"
    ],
    
    "category": [
        *["Factory Automation"]*20,
        *["IoT"]*20,
        *["Process Control"]*20,
        *["Safety and Standards"]*20,
        *["General"]*20
    ]
}

third_data = pd.DataFrame(additional_data)

In [107]:
automation_general_knowledge = {
    "text": [
        # Automation General Knowledge (20)
        "Fundamental components of a PLC system",
        "Difference between open-loop and closed-loop control",
        "Purpose of ladder logic in industrial automation",
        "Basic principles of PID controller operation",
        "Common industrial communication protocols (e.g., Modbus, Profibus)",
        "Safety standards for machine guarding (ISO 13857)",
        "Types of industrial sensors and their applications",
        "Concept of human-machine interface (HMI) design",
        "Principles of pneumatic system operation",
        "Basics of motor control circuits",
        "Importance of failsafe design in automation",
        "Types of industrial robots and their configurations",
        "Fundamentals of SCADA system architecture",
        "Purpose of safety instrumented systems (SIS)",
        "Basic electrical symbols used in control diagrams",
        "Concept of machine vision in quality inspection",
        "Principles of variable frequency drive (VFD) operation",
        "Types of industrial actuators and their applications",
        "Basics of hydraulic system maintenance",
        "Fundamentals of industrial networking topologies"
    ],
    
    "category": [
        *["General"]*20
    ]
}

forth_data = pd.DataFrame(automation_general_knowledge)

In [108]:
thai_training_data = {
    "text": [
        # Factory Automation (20)
        "การเชื่อมโลหะด้วยหุ่นยนต์แบบใช้ระบบวิทัศน์ในสายการประกอบยานยนต์",
        "การวางแผนเส้นทาง AGV ด้วยเซนเซอร์ LiDAR",
        "ระบบตรวจสอบการสึกหรอของเครื่องมือ CNC แบบอัตโนมัติ",
        "การผสานหุ่นยนต์ร่วมงานกับระบบ PLC แบบเดิม",
        "ระบบตรวจสอบข้อบกพร่องพื้นผิวด้วยการมองเห็นอัตโนมัติ",
        "กลไกเปลี่ยนอุปกรณ์จับชิ้นงานของหุ่นยนต์",
        "เทคนิคการซิงโครไนซ์ความเร็วสายพานอัตโนมัติ",
        "ขั้นตอนการปรับเทียบเซลล์ทำงานของหุ่นยนต์อุตสาหกรรม",
        "การเพิ่มประสิทธิภาพการผลิตไลน์บรรจุภัณฑ์อัตโนมัติ",
        "ระบบให้บริการเครื่อง CNC แบบอัตโนมัติ",
        "หุ่นยนต์ขัดเงาแบบควบคุมแรงกดอัตโนมัติ",
        "ระบบวัดพิกัดด้วยเครื่อง CMM แบบอัตโนมัติ",
        "หุ่นยนต์กำจัดคราบด้วยระบบวิทัศน์สำหรับชิ้นส่วนหล่อ",
        "ระบบตรวจสอบอายุเครื่องมือด้วยคลื่นเสียง",
        "การปรับเส้นทางปะกาวด้วยหุ่นยนต์อัตโนมัติ",
        "ระบบจัดการจราจร AGV ในโรงงาน",
        "ระบบคัดแยกด้วยหุ่นยนต์และระบบวิทัศน์",
        "อัลกอริทึมชดเชยอุณหภูมิสปินเดิลอัตโนมัติ",
        "การติดตามรอยเชื่อมด้วยเลเซอร์ในหุ่นยนต์เชื่อม",
        "ระบบจับชิ้นงานไร้ฟิกซ์เจอร์อัตโนมัติ",

        # IoT (20)
        "ระบบตรวจสอบการสั่นสะเทือนแบบไร้สายสำหรับตลับลูกปืนปั๊ม",
        "การสร้างฝาแฝดดิจิทัลเพื่อเพิ่มประสิทธิภาพพลังงาน HVAC",
        "ระบบตรวจสอบระดับถังด้วย LoRaWAN",
        "การวิเคราะห์การสั่นสะเทือนแบบเรียลไทม์ที่ Edge",
        "การบำรุงรักษาเชิงพยากรณ์สำหรับคอมเพรสเซอร์ด้วย IoT",
        "เครือข่ายเซนเซอร์แบบ Wireless Mesh ในโรงงาน",
        "การใช้งานโปรโตคอล MQTT ในเกตเวย์ IIoT",
        "การวิเคราะห์ข้อมูลเซนเซอร์อุณหภูมิบนคลาวด์",
        "การควบคุมคุณภาพด้วยเซนเซอร์หลายประเภท",
        "การเชื่อมต่อระบบวิทัศน์เครื่องด้วย 5G",
        "ระบบติดตามทรัพย์สินด้วย BLE ในคลังสินค้า",
        "การตรวจสอบการใช้พลังงานมอเตอร์ผ่าน IoT",
        "เซนเซอร์อัจฉริยะสำหรับระบบไฮดรอลิก",
        "การตรวจสอบเครื่องทำความเย็นทางไกลผ่าน IoT",
        "ระบบตรวจสอบอุปกรณ์เกษตรด้วย LPWAN",
        "การวิเคราะห์สภาพใบพัดกังหันลมด้วยการสั่นสะเทือน",
        "ระบบจัดการหล่อลื่นอัจฉริยะด้วย IoT",
        "การควบคุมแสงสว่างอัจฉริยะด้วยเซนเซอร์ตรวจจับการเคลื่อนไหว",
        "การตรวจสอบการกัดกร่อนในท่อเคมีแบบไร้สาย",
        "ระบบตรวจจับการรั่วในระบบลมอัดด้วย IoT",

        # Process Control (20)
        "การปรับแต่ง PID ขั้นสูงสำหรับเครื่องปฏิกรณ์เอกโซเทอร์มิก",
        "การควบคุมแบบพยากรณ์สำหรับคอลัมน์กลั่น",
        "การตั้งค่าควบคุมแบบ Cascade สำหรับเครื่องแลกเปลี่ยนความร้อน",
        "การชดเชยการติดขัดของวาล์วควบคุมความดัน",
        "การควบคุมหลายตัวแปรสำหรับเครื่องปฏิกรณ์แบบฟลูอิไดซ์เบด",
        "กลยุทธ์การควบคุมแบบปรับตัวสำหรับการควบคุม pH",
        "การชดเชยเวลาตายในวงจรควบคุมอุณหภูมิ",
        "ระบบป้องกันปั๊มด้วยการควบคุม Override",
        "การควบคุมกระบวนการแบบสถิติสำหรับการผลิตเป็นชุด",
        "การควบคุมอัตราส่วนในกระบวนการผสม",
        "การควบคุม Feedforward สำหรับหม้อต้ม",
        "การควบคุมแบบเมทริกซ์ไดนามิกสำหรับเครื่องปฏิกรณ์พอลิเมอไรเซชัน",
        "การควบคุม Split-range สำหรับระบบความดัน",
        "กลยุทธ์ควบคุมแบบไม่เชิงเส้นสำหรับเครื่องปฏิกรณ์ชีวภาพ",
        "การควบคุมผลึกแบบแบตช์ด้วยวิธี Optimal Control",
        "การปรับจูนวาล์วโพสิชันเนอร์สำหรับการตอบสนองเร็ว",
        "การควบคุมความชื้นด้วย Fuzzy Logic",
        "การปรับ Gain ตามฤดูกาลสำหรับกระบวนการอบความร้อน",
        "การควบคุมแบบแยกคู่สำหรับถาดกลั่น",
        "การควบคุม Robust สำหรับกระบวนการเอกโซเทอร์มิกไม่เสถียร",

        # Safety and Standards (20)
        "การใช้งานระบบความปลอดภัย SIL-2 สำหรับเครื่องกด",
        "การติดตั้งเซนเซอร์ตามมาตรฐาน ATEX ในโซน 1",
        "การติดตั้งม่านแสงปลอดภัยสำหรับเซลล์หุ่นยนต์",
        "การรับรอง ISO 13849 สำหรับเครื่องบรรจุภัณฑ์",
        "การตรวจสอบระบบหยุดฉุกเฉินแบบซ้ำซ้อน",
        "การประเมินความปลอดภัยเชิงหน้าที่สำหรับอุปกรณ์ CNC",
        "วิธีการประเมินความเสี่ยงตาม ISO 12100",
        "การเขียนโปรแกรม PLC ความปลอดภัยสำหรับระบบดับเพลิง",
        "การปฏิบัติตามมาตรฐาน NFPA 79 สำหรับแผงควบคุม",
        "มาตรฐานเอกสารขั้นตอน Lockout-Tagout",
        "การตรวจสอบเวลาตอบสนองฟังก์ชันความปลอดภัย IEC 62061",
        "การคำนวณระยะปลอดภัยตาม ISO 13855",
        "การทดสอบแรงกดปุ่มหยุดฉุกเฉิน",
        "วิธีการตรวจจับการเชื่อมต่อรีเลย์ความปลอดภัย",
        "การวิเคราะห์ระดับประสิทธิภาพความปลอดภัย (PL)",
        "การตรวจสอบโปรแกรม PLC ความปลอดภัย",
        "การประยุกต์ใช้แผนภูมิความเสี่ยงเครื่องจักร",
        "การตรวจสอบความสมบูรณ์ของเครือข่ายความปลอดภัย",
        "การทดสอบระบบระบายอากาศฉุกเฉิน",
        "การตรวจสอบการให้คะแนนตู้กันระเบิด",

        # General (20)
        "วิธีการรีเซ็ตรหัสผ่านบัญชีผู้ใช้",
        "ปัญหาในการเข้าถึงใบแจ้งหนี้เดิม",
        "แอพมือถือหยุดทำงานบน iOS 17.2",
        "วิธีการขอเพิ่มขีดจำกัด API",
        "ไม่สามารถดาวน์โหลดไฟล์ซอฟต์แวร์ที่ซื้อแล้ว",
        "วิธีการอัปเดตที่อยู่การเรียกเก็บเงิน",
        "ปัญหาการตั้งค่าการยืนยันตัวสองขั้นตอน",
        "วิธีการรายงานเนื้อหาไม่เหมาะสมในฟอรัม",
        "แก้ไขปัญหาการส่งออกไฟล์ PDF ล้มเหลว",
        "วิธีการขยายระยะเวลาทดลองใช้",
        "การรวมบัญชีผู้ใช้หลายบัญชี",
        "ปัญหาเซสชันล็อกอินหมดอายุ",
        "วิธีการขอใบรับรองความถูกต้องซอฟต์แวร์",
        "กู้คืนไฟล์โครงการที่ถูกลบจากคลาวด์",
        "แก้ไขปัญหาการชำระเงินต่ออายุสมาชิก",
        "วิธีการส่งออกประวัติข้อมูลจากแดชบอร์ด",
        "เปลี่ยนการตั้งค่าการแจ้งเตือนในพอร์ทัล",
        "แก้ไขปัญหาความเข้ากันได้กับระบบปฏิบัติการเก่า",
        "วิธีการโอนใบอนุญาตระหว่างอุปกรณ์",
        "ทำความเข้าใจข้อตกลงระดับบริการ (SLA)"
    ],
    
    "category": [
        *["Factory Automation"]*20,
        *["IoT"]*20,
        *["Process Control"]*20,
        *["Safety and Standards"]*20,
        *["General"]*20
    ]
}

thai_data = pd.DataFrame(thai_training_data)

In [109]:

# Test Data (from previous response)
test_data = {
    "text": [
        # Factory Automation (20)
        "Vision-guided robotic welding in automotive assembly lines",
        "Optimizing AGV path planning using LiDAR sensors",
        "Automated CNC tool wear detection systems",
        "Integrating collaborative robots with legacy PLC systems",
        "Automated visual inspection for surface defects",
        "Robotic end-effector tool changing mechanisms",
        "Automated conveyor speed synchronization techniques",
        "Industrial robot workcell calibration procedures",
        "Automated packaging line throughput optimization",
        "Machine tending automation for CNC machining centers",
        "Robotic palletizing systems for warehouse automation",
        "AI-based predictive maintenance for robotic arms",
        "Automated guided vehicle (AGV) battery management systems",
        "Smart conveyor belt fault detection using vibration sensors",
        "Automated screwdriving systems with torque verification",
        "Industrial robot path optimization using digital twins",
        "Automated quality control with 3D scanning technology",
        "Flexible manufacturing system (FMS) scheduling algorithms",
        "Automated deburring systems with force feedback control",
        "Vision-based bin picking for random part orientation",

        # IoT (20)
        "Wireless vibration monitoring for pump bearings",
        "Implementing digital twins for HVAC energy optimization",
        "LoRaWAN-based tank level monitoring solutions",
        "Edge computing for real-time vibration analysis",
        "IoT-enabled predictive maintenance for compressors",
        "Wireless mesh networks for factory sensor deployment",
        "MQTT protocol implementation for IIoT gateways",
        "Cloud-based analytics for temperature sensor data",
        "Predictive quality control using multisensor fusion",
        "5G-enabled machine vision system connectivity",
        "Bluetooth Low Energy (BLE) asset tracking in warehouses",
        "IoT-based energy consumption monitoring for motors",
        "Smart sensors for predictive maintenance of hydraulic systems",
        "Remote monitoring of industrial chillers via IoT",
        "LPWAN solutions for agricultural equipment monitoring",
        "Vibration-based condition monitoring for wind turbines",
        "IoT-driven lubrication management for conveyor systems",
        "Smart lighting control using occupancy sensors",
        "Wireless corrosion monitoring in chemical pipelines",
        "IoT-based leak detection in compressed air systems",

        # Process Control (20)
        "Advanced PID tuning for exothermic reactors",
        "Model predictive control for distillation columns",
        "Cascade control configuration for heat exchangers",
        "Pressure control valve stiction compensation",
        "Multivariable control for fluidized bed reactors",
        "Adaptive control strategies for pH regulation",
        "Dead-time compensation in temperature loops",
        "Override control for pump protection systems",
        "Statistical process control for batch manufacturing",
        "Ratio control optimization for mixing processes",
        "Feedforward control for boiler temperature regulation",
        "Dynamic matrix control for polymerization reactors",
        "Split-range control implementation for pressure management",
        "Nonlinear control strategies for bioreactors",
        "Optimal control of batch crystallization processes",
        "Real-time optimization of chemical dosing systems",
        "Fuzzy logic control for humidity regulation",
        "Gain scheduling for nonlinear heat treatment processes",
        "Decoupling control for interacting distillation trays",
        "Robust control design for unstable exothermic processes",

        # Safety and Standards (20)
        "Implementing SIL-2 safety systems for press machines",
        "ATEX-compliant sensor installation in Zone 1",
        "Safety light curtain integration for robot cells",
        "ISO 13849 certification for packaging machinery",
        "Emergency stop circuit redundancy validation",
        "Functional safety assessment for CNC equipment",
        "Risk assessment methodology per ISO 12100",
        "Safety PLC programming for emergency shutdown",
        "NFPA 79 compliance for industrial control panels",
        "Lockout-tagout procedure documentation standards",
        "IEC 62061 compliance for machinery safety systems",
        "Safety interlock verification procedures for ovens",
        "Emergency ventilation system activation protocols",
        "ANSI B11 standards implementation for presses",
        "Safety door switch integration with PLC logic",
        "Burn risk mitigation in automated welding stations",
        "Safety distance calculations for robotic workcells",
        "Explosion-proof enclosure ratings for hazardous areas",
        "Safety relay configuration for emergency stops",
        "Arc flash protection system commissioning",

        # General (20)
        "How to reset my account password?",
        "Trouble accessing previous order invoices",
        "Mobile app crashing on iOS 17.2",
        "Requesting API rate limit increase",
        "Can't download purchased software files",
        "How to update billing address information?",
        "Two-factor authentication setup issues",
        "Report inappropriate forum post content",
        "Troubleshooting PDF export failures",
        "Request extension for trial period",
        "How to merge multiple user accounts?",
        "Troubleshooting login session timeout issues",
        "Requesting certificate of software authenticity",
        "Recovering deleted project files from cloud",
        "Subscription renewal payment failure resolution",
        "Exporting historical data logs from dashboard",
        "Changing notification preferences in web portal",
        "Resolving compatibility issues with legacy OS",
        "Steps to transfer license between devices",
        "Understanding service level agreement terms",

        # Edge Cases (10)
        "IoT security protocols for PLC networks",
        "Automated safety shutdown systems",
        "Wireless control valve monitoring",
        "Robot collision detection algorithms",
        "Predictive maintenance using vibration",
        "Safety-rated network protocols",
        "Automated calibration documentation",
        "Cloud-based process optimization",
        "Emergency stop programming guidelines",
        "Mobile app for equipment monitoring"
    ],
    
    "true_category": [
        *["Factory Automation"]*20,
        *["IoT"]*20,
        *["Process Control"]*20,
        *["Safety and Standards"]*20,
        *["General"]*20,
        "IoT", "Safety and Standards", "IoT", "Factory Automation", "IoT",
        "Safety and Standards", "Factory Automation", "IoT", "Safety and Standards", "IoT"
    ]
}

automation_general_test = {
    "text": [
        # Automation General Knowledge (20)
        "PLC scan cycle sequence and timing considerations",
        "Difference between relay logic and PLC-based control",
        "Purpose of HART protocol in process instrumentation",
        "Key components of a SCADA system architecture",
        "Fundamental difference between fail-safe and fault-tolerant systems",
        "Basic principles of 4-20mA current loop operation",
        "Purpose of safety relays in emergency stop circuits",
        "Common types of industrial proximity sensors",
        "Concept of 'sinking' vs 'sourcing' in digital I/O",
        "Basic functionality of a PID controller's integral term",
        "Purpose of IP ratings in industrial equipment",
        "Difference between absolute and incremental encoders",
        "Basic operation principle of a variable frequency drive",
        "Purpose of ladder logic seal-in circuits",
        "Key differences between Ethernet/IP and PROFINET",
        "Fundamental concept of machine guarding hierarchy",
        "Purpose of purge and pressurization systems",
        "Basic types of pneumatic actuators and their applications",
        "Concept of 'duty cycle' in motor operations",
        "Purpose of galvanic isolation in signal conditioning",
        "อะไรคือหุ่นยนต์"
    ],
    
    "true_category": [
        *["General"]*20,
        *["Factory Automation"]*1
    ]
}


In [110]:
thai_test_data = {
    "text": [
        # Factory Automation (20)
        "ระบบปรับแรงกดอัตโนมัติสำหรับหุ่นยนต์ขัดเงา",
        "การผสานเครื่องวัดพิกัด CMM ในสายการผลิต",
        "หุ่นยนต์กำจัดคราบด้วยระบบวิทัศน์สำหรับชิ้นส่วนหล่อ",
        "การตรวจสอบอายุเครื่องมือด้วยคลื่นเสียง",
        "การปรับเส้นทางปะกาวด้วยหุ่นยนต์อัตโนมัติ",
        "ระบบจัดการจราจร AGV ในโรงงาน",
        "ระบบคัดแยกด้วยหุ่นยนต์และระบบวิทัศน์",
        "อัลกอริทึมชดเชยอุณหภูมิสปินเดิลอัตโนมัติ",
        "การติดตามรอยเชื่อมด้วยเลเซอร์ในหุ่นยนต์เชื่อม",
        "ระบบจับชิ้นงานไร้ฟิกซ์เจอร์อัตโนมัติ",
        "การตรวจสอบแผ่นวงจร PCB ด้วยระบบวิทัศน์",
        "การปรับรูปแบบการวางพาเลทด้วยหุ่นยนต์",
        "ระบบชดเชยออฟเซ็ตเครื่องมือ CNC",
        "การติดตามรอยเชื่อมด้วยระบบวิทัศน์",
        "ระบบตรวจสอบการไหลของสารหล่อลื่น",
        "การปรับเส้นทางตัดด้วยน้ำแรงดันสูง",
        "การแปลงระบบพิกัดสำหรับหุ่นยนต์",
        "ระบบตรวจจับการติดขัดสายพาน",
        "ระบบเปลี่ยนแม่พิมพ์อัตโนมัติ",
        "การตรวจสอบชิ้นงานด้วยระบบวิทัศน์",

        # IoT (20)
        "การตรวจสอบอุณหภูมิตลับลูกปืนแบบไร้สาย",
        "ระบบตรวจจับการเกิดโพรงอากาศในปั๊ม",
        "การตรวจสอบระดับถังเคมีด้วย NB-IoT",
        "การวิเคราะห์ความล้มเหลวของขดลวดมอเตอร์",
        "การตรวจสอบการสั่นสะเทือนแบบไร้สาย",
        "ระบบตรวจสอบประสิทธิภาพสตีมทราป",
        "การบังคับใช้โซนปลอดภัยด้วย BLE",
        "การตรวจสอบอัตราการกัดกร่อนแบบไร้สาย",
        "การสร้างฝาแฝดดิจิทัลสำหรับปั๊ม",
        "ระบบตรวจสอบการสึกหรอสายพาน",
        "การตั้งค่าเครือข่ายเครื่องส่งสัญญาณความดัน",
        "การตรวจจับรูปแบบความร้อนผิดปกติ",
        "การตรวจสอบการปนเปื้อนของไฮดรอลิก",
        "การตรวจสอบแรงบิดแบบไร้สาย",
        "การตรวจสอบสิ่งแวดล้อมด้วย LPWAN",
        "ระบบจัดการการหล่อลื่นแบบพยากรณ์",
        "การตรวจสอบการไหลแบบไร้สาย",
        "การวิเคราะห์คุณภาพไฟฟ้าแบบเรียลไทม์",
        "การพยากรณ์การเสื่อมสภาพของปั๊ม",
        "การตรวจสอบสัญญาณเอ็นโคเดอร์แบบไร้สาย",

        # Process Control (20)
        "การควบคุมแบบปรับตัวสำหรับเครื่องแลกเปลี่ยนความร้อน",
        "การควบคุมอัตราส่วนในกระบวนการทำให้เป็นกลาง",
        "การชดเชยเวลาตายด้วย Smith Predictor",
        "การควบคุมหลายตัวแปรสำหรับการกลั่น",
        "ระบบป้องกันการกระเพื่อมของคอมเพรสเซอร์",
        "การควบคุม Feedforward สำหรับวัตถุดิบ",
        "การปรับจูน PID แบบไม่เชิงเส้นสำหรับ pH",
        "การเพิ่มประสิทธิภาพเครื่องปฏิกรณ์แบบเรียลไทม์",
        "การจัดการข้อจำกัดในระบบ MPC",
        "การตรวจจับการติดขัดของวาล์ว",
        "การกรองแบบปรับตัวสำหรับการวัดการไหล",
        "การแยกคู่ควบคุมระดับ",
        "การเปลี่ยนโหมดควบคุมอย่างราบรื่น",
        "การปรับ Gain ตามฤดูกาล",
        "กลยุทธ์ Anti-windup สำหรับวาล์ว",
        "การควบคุมแบบทนต่อข้อผิดพลาด",
        "การควบคุมโดยใช้ Soft Sensor",
        "การออกแบบ Robust Control",
        "การปรับจูน Cascade Control",
        "การควบคุม Split-range สำหรับระบบทำความร้อน",

        # Safety and Standards (20)
        "การตรวจสอบ PL ตาม ISO 13849",
        "การตรวจสอบการจัดตำแหน่งม่านแสง",
        "วิธีการตรวจสอบ SIL ตาม IEC 62061",
        "แนวทางการติดตั้งอุปกรณ์ ATEX Zone 2",
        "การตรวจสอบประสิทธิภาพวงจรหยุดฉุกเฉิน",
        "การวิเคราะห์ความครอบคลุมโค้ด PLC",
        "ข้อกำหนดอุปกรณ์ Interlock ตาม ISO 14119",
        "การวัดเวลาตอบสนอง Safety Mat",
        "การกำหนด Safety Category ตาม EN 954-1",
        "รายการตรวจสอบขั้นตอน Lockout-Tagout",
        "การประเมิน Diagnostic Coverage ของรีเลย์",
        "การคำนวณระยะปลอดภัย ISO 13855",
        "การทดสอบ Watchdog Timer",
        "การวิเคราะห์ความซ้ำซ้อนวงจรหยุดฉุกเฉิน",
        "การตรวจสอบโปรโตคอลเครือข่ายความปลอดภัย",
        "การตรวจสอบไฟฟ้าตาม IEC 60204",
        "วิธีการตรวจสอบสัญญาณเอ็นโคเดอร์",
        "รายงานการตรวจสอบ Performance Level",
        "การทดสอบ Fault Injection",
        "การวัดแรงกดปุ่มหยุดฉุกเฉิน",

        # General (20)
        "วิธีการตั้งค่าสิทธิ์ API",
        "แก้ไขปัญหาการล่าช้าในการซิงค์ข้อมูล",
        "การแก้ไขข้อผิดพลาดการตรวจสอบใบรับรอง",
        "วิธีการส่งออกรายงาน Audit Trail",
        "ปัญหาความเข้ากันได้ของแอพกับ iOS 18",
        "แก้ไขปัญหา Connection Pool ฐานข้อมูล",
        "วิธีการใช้งานการยืนยันตัวแบบกำหนดเอง",
        "แก้ไขปัญหาความไม่ตรงกันของ Timestamp",
        "การตั้งค่าการสำรองข้อมูลอัตโนมัติ",
        "แก้ไขปัญหา Memory Leak",
        "วิธีการเปิดใช้งาน MFA",
        "แก้ไขปัญหาการจัดรูปแบบ CSV",
        "การตั้งค่าการเข้าถึงตามบทบาท",
        "แก้ไขข้อผิดพลาด Rate Limit API",
        "วิธีการสร้างรายงานสุขภาพระบบ",
        "แก้ไขปัญหา SSL Handshake",
        "การตั้งค่านโยบายการเก็บข้อมูล",
        "แก้ไขปัญหาการอัปเดตเฟิร์มแวร์",
        "วิธีการขอเอกสาร Schema API",
        "แก้ไขปัญหาการแปลงเขตเวลา"
    ],
    
    "true_category": [
        *["Factory Automation"]*20,
        *["IoT"]*20,
        *["Process Control"]*20,
        *["Safety and Standards"]*20,
        *["General"]*20
    ]
}
thai_test_data = pd.DataFrame(thai_test_data)

In [111]:

full_data = pd.concat([first_data, second_data, third_data, forth_data, thai_data], ignore_index=True)

# Training Data (using your augmented data)
train_df = pd.DataFrame(full_data)
train_df.to_csv('train.csv')

test_1 = pd.DataFrame(test_data)
test_2 = pd.DataFrame(automation_general_test)

test_df = pd.concat([test_1, test_2, thai_test_data] , ignore_index=True)

X_test = test_df['text']
y_test = test_df['true_category']

# Create and train model
model = make_pipeline(TfidfVectorizer(
                                      tokenizer=custom_tokenizer
                                      ), 
                      MultinomialNB()
                      )
model.fit(train_df['text'], train_df['category'])

# Threshold configuration
CONFIDENCE_THRESHOLD = 0.2

# Make predictions with confidence check
predicted_probabilities = model.predict_proba(X_test)
predicted_categories = []

for probs in predicted_probabilities:
    max_prob = max(probs)
    if max_prob < CONFIDENCE_THRESHOLD:
        predicted_categories.append("General")
    else:
        predicted_categories.append(model.classes_[np.argmax(probs)])

# Evaluate performance
print("Classification Report:")
print(classification_report(y_test, predicted_categories))

print("\nConfusion Matrix:")
print(confusion_matrix(y_test, predicted_categories, labels=model.classes_))

# Print individual results with confidence scores
print("\nDetailed Predictions:")
for msg, true, pred, probs in zip(X_test, y_test, predicted_categories, predicted_probabilities):
    confidence = max(probs)
    decision = "✓" if true == pred else f"✗ (True: {true})"
    print(f"{decision} | {pred:<20} | Conf: {confidence:.2f} | Text: {msg}")

Classification Report:
                      precision    recall  f1-score   support

  Factory Automation       0.83      0.91      0.87        43
             General       0.95      0.92      0.93        60
                 IoT       0.91      0.87      0.89        45
     Process Control       0.95      0.88      0.91        40
Safety and Standards       0.83      0.88      0.85        43

            accuracy                           0.89       231
           macro avg       0.89      0.89      0.89       231
        weighted avg       0.90      0.89      0.89       231


Confusion Matrix:
[[39  0  3  0  1]
 [ 1 55  0  1  3]
 [ 2  1 39  1  2]
 [ 1  1  1 35  2]
 [ 4  1  0  0 38]]

Detailed Predictions:
✓ | Factory Automation   | Conf: 0.64 | Text: Vision-guided robotic welding in automotive assembly lines
✓ | Factory Automation   | Conf: 0.47 | Text: Optimizing AGV path planning using LiDAR sensors
✓ | Factory Automation   | Conf: 0.57 | Text: Automated CNC tool wear detection sys

c:\Users\User\Desktop\Git\CapStone-Project\.venv\lib\site-packages\sklearn\feature_extraction\text.py:517: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [112]:
joblib.dump(model, "text_classification.pkl")

['text_classification.pkl']

In [113]:
loaded_model = joblib.load("text_classification.pkl")

In [114]:
# Sample text data (new messages that you want to classify)
new_text = [
    "IoT security protocols for PLC networks",
    "Automated safety shutdown systems",
    "Wireless control valve monitoring",
    "Robot collision detection algorithms",
    "Predictive maintenance using vibration",
    "Safety-rated network protocols",
    "Automated calibration documentation",
    "Cloud-based process optimization",
    "Emergency stop programming guidelines",
    "Mobile app for equipment monitoring"
]

# Make predictions
predictions = loaded_model.predict(new_text)

# Display the results
for msg, category in zip(new_text, predictions):
    print(f"Message: {msg} -> Predicted Category: {category}")


Message: IoT security protocols for PLC networks -> Predicted Category: IoT
Message: Automated safety shutdown systems -> Predicted Category: Factory Automation
Message: Wireless control valve monitoring -> Predicted Category: IoT
Message: Robot collision detection algorithms -> Predicted Category: Factory Automation
Message: Predictive maintenance using vibration -> Predicted Category: IoT
Message: Safety-rated network protocols -> Predicted Category: Safety and Standards
Message: Automated calibration documentation -> Predicted Category: Factory Automation
Message: Cloud-based process optimization -> Predicted Category: Process Control
Message: Emergency stop programming guidelines -> Predicted Category: Safety and Standards
Message: Mobile app for equipment monitoring -> Predicted Category: General
